<a href="https://colab.research.google.com/github/ivanachen1/LinearRegression/blob/main/LinnearRegressionAux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [6]:
def carga_datos():
  """
  Esta funcion carga el dataframe
  """

  list_dataframes = []

  for i in range(52,63,1):
    if i in [50,51,54]:
      continue
    elif i == 63:
      break
    else:
      if i in [52,53,55,56,57]:
        data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Machine Learning/VENTA_DIARIA_6{}.csv".format(i),sep=",")
      else:
        data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Machine Learning/VENTA_DIARIA_6{}.csv".format(i),sep=",") 
      
      list_dataframes.append(data)
    
  dataframe = pd.concat(list_dataframes)

  return dataframe

In [8]:
def cambio_formatos(dataframe):
  """
  Esta funcion cambia el formato de los campos del 
  dataframe
  """
  dataframe["cantidad"] = dataframe["cantidad"].astype(int)
  dataframe["mes"] = dataframe["mes"].astype(int)
  dataframe["anio"] = dataframe["anio"].astype(int)
  dataframe["operating_unit_id"] = dataframe["operating_unit_id"].astype(int)
  dataframe["product_id"] = dataframe["product_id"].astype(int)

  return dataframe

In [9]:
def campos_datetime(dataframe):
  """
  Esta funcion crea los campos en formato datetime
  """

  dataframe["Fecha_format_datetime"] = pd.to_datetime(dataframe['fecha'])
  dataframe.drop(["fecha","mes","anio","tipo"],axis = 1, inplace= True)
  dataframe = dataframe[dataframe["cantidad"] > 0]

  dataframe.sort_values(by = "Fecha_format_datetime",inplace = True)
  #df['year'] = pd.DatetimeIndex(df['ArrivalDate']).year
  dataframe["day"] = pd.DatetimeIndex(dataframe["Fecha_format_datetime"]).day
  dataframe["month"] = pd.DatetimeIndex(dataframe["Fecha_format_datetime"]).month
  dataframe["year"] = pd.DatetimeIndex(dataframe["Fecha_format_datetime"]).year
  dataframe["week"] = pd.DatetimeIndex(dataframe["Fecha_format_datetime"]).week
  dataframe["week2"] = pd.DatetimeIndex(dataframe["Fecha_format_datetime"]).strftime('%U_%Y')
  dataframe["Quarter"] = pd.DatetimeIndex(dataframe["Fecha_format_datetime"]).quarter
  dataframe = dataframe.assign(week2= dataframe['Fecha_format_datetime'].dt.strftime('%Y_%U')).sort_values('week2')
  dataframe["Semester"] = np.where((dataframe["Quarter"] == 1) | (dataframe["Quarter"] == 2),1,2)
  dataframe.sort_values(by = "week2",inplace = True)

  return dataframe

In [10]:
def crear_accumulative_week(dataframe):
  """
  Crea el campo de conteo de semanas de un año para el otro
  """
  lista_anios = dataframe["anio"].unique()

  field = []
  for row in dataframe.itertuples():
    year = row[7]
    if year == min(lista_anios):
      week = row[8]
      field.append(week)
    else:
      diference = year - min(lista_anios)
      calculo = (row[8] + 52) * diference
      field.append(calculo)


    field = np.array(field)
    dataframe["acumulative_week"] = field

  return dataframe

In [11]:
def agregar_semanas_cantidades_cero(dataframe,lista_de_articulos):
  """
  Esta funcion agrega las semanas restantes al dataframe en 
  base a los articulos suministrados en lista pasado y devuelve el 
  mismo completo
  """

  lista_numeros_semana = list(dataframe["acumulative_week"].unique())
  lista_weeks_ordenadas = []

  for i in range(1,len(lista_numeros_semana)+1,1):
    if i in lista_numeros_semana:
      continue
  else:
    lista_numeros_semana.append(i)
  
  dataframe_to_predict = []
  i = 0
  df2 = dataframe.drop(labels = ["operating_unit_id","Fecha_format_datetime","day","month","year","week","week2","Quarter","Semester"],axis=1)
  
  for articulo in lista_articulos:
    data_temporal = df2[df2["product_id"] == articulo]
    dataframe_to_predict.append(data_temporal)
  
  for semana in lista_numeros_semana:
    if semana not in list(data_temporal["acumulative_week"]):
      data = pd.DataFrame({"product_id":articulo,"acumulative_week":semana,"cantidad":0},index = [i])
      i = i+1     
    dataframe_to_predict.append(data)

  dataframe_to_predict = pd.concat(dataframe_to_predict)

  return dataframe_to_predict

In [12]:
def predecir(dataframe,lista_articulos):
  """
  Esta funcion itera el dataframe que se le pasa como parametro y te devuelve una tabla con
  los articulos y los R cuadrados
  """
  import statsmodels.api as sm
  import numpy as np
  import pandas as pd
  
  dataframe_prediccion = []

  i = 0
  for articulo in lista_articulos:
    data_prueba = dataframe_to_predict[dataframe_to_predict["product_id"] == articulo]
    Y = np.array(data_prueba["cantidad"])
    X = np.array(data_prueba["acumulative_week"])
    model = sm.OLS(Y,X)
    results = model.fit()
    R2 = results.rsquared
    i = i+1


    data = {"Articulo":articulo,"R square":R2}
    i = i+1     
    dataframe_prediccion.append(data)
    


  dataframe_prediccion = pd.DataFrame(dataframe_prediccion)

  return dataframe_prediccion